In [1]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch import nn
import tensorflow as tf

In [4]:
#Importando dataset Acelerômetro
caminho = "SmartWatch/accel/"
arquivos = [arquivo for arquivo in os.listdir(caminho) if arquivo.endswith('.txt')]
accl = []
for i in arquivos:
  acc = pd.read_table(caminho+i, header=None, sep=",")
  accl.append(acc)
accel = pd.concat(accl, ignore_index=True)
accel.columns = ['pessoa', 'atividade', 'timestamp', 'x', 'y', 'z']
accel = accel.sort_values(by=['pessoa', 'atividade'])
accel = accel.reset_index(drop=True)
accel['z'] = accel['z'].replace(';', '', regex=True)

,0,1,2,3,4,5
0,1650,A,2428057133445480,-1.279851,-5.187936,-2.3116016;
1,1650,A,2428057182945480,-3.302951,-6.823175,-4.011485;
2,1650,A,2428057232445480,-2.876783,-8.365042,-7.171831;
3,1650,A,2428057281945480,-2.254291,-8.566154,-8.153454;
4,1650,A,2428057331445480,-3.264644,-10.953174,-8.826224;
...,...,...,...,...,...,...
66587,1650,S,2426184421424592,1.486500,-13.302334,5.355231;
66588,1650,S,2426184471411932,4.017171,-13.146711,5.1182046;
66589,1650,S,2426184521399272,4.412214,-15.318253,3.3704374;
66590,1650,S,2426184571385262,4.426579,-13.012636,2.5755625;


In [5]:
#Importando dataset Giroscópio
caminho = "SmartWatch/gyro/"
arquivos = [arquivo for arquivo in os.listdir(caminho) if arquivo.endswith('.txt')]
g = []
for i in arquivos:
  giro = pd.read_table(caminho+i, header=None, sep=",")
  g.append(giro)
giro = pd.concat(g, ignore_index=True)
giro.columns = ['pessoa', 'atividade', 'timestamp', 'x', 'y', 'z']
giro = giro.sort_values(by=['pessoa', 'atividade'])
giro = giro.reset_index(drop=True)
giro['z'] = giro['z'].replace(';', '', regex=True)

In [6]:
accel['z'] = pd.to_numeric(accel['z'], errors='coerce')
giro['z'] = pd.to_numeric(giro['z'], errors='coerce')

In [7]:

def aceleracao_total(x, y, z):
  t = (y**2 + x**2 + z**2)**0.5
  return t
def giro_total(x,y,z, x1, y1, z1):
  t = ((x**2)+(x1**2) - 2*x*x1*(np.sin(y)*np.sin(y1)*np.cos((z-z1))+np.cos(y)*np.cos(y1)))**0.5
  return t

In [8]:
def transacc(x):
  for j in range(0, 1200, 20):
    valor = 0;
    for i in range(0, 20, 1):
      valor = valor + aceleracao_total(x.iloc[j+i, 3], x.iloc[(j+i), 4], x.iloc[(j+i), 5])
    dadospa.append(valor)
    atividade.append(x.iloc[j+i, 1])

def transgiro(x):
  for j in range(0, 1200, 20):
    valor= 0;
    for i in range(0,19,1):
      valor = valor + giro_total(x.iloc[(j+i),3], x.iloc[(j+i),4], x.iloc[(j+i),5], x.iloc[(j+i+1),3], x.iloc[(j+i+1),4], x.iloc[(j+i+1),5])
    dadospg.append(valor)

In [9]:
dadospa = []
dadospg = []
atividade = []
for i in range(1200,len(giro), 1200):
  transacc(accel[(i-1200):i])
  transgiro(giro[(i-1200):i])

C:\Users\Paulo Pires\AppData\Local\Temp\ipykernel_33972\2909828199.py:5: RuntimeWarning: invalid value encountered in scalar power
  t = ((x**2)+(x1**2) - 2*x*x1*(np.sin(y)*np.sin(y1)*np.cos((z-z1))+np.cos(y)*np.cos(y1)))**0.5


In [10]:
print(len(dadospa))
print(len(dadospg))
print(len(atividade))
print(171960/60)

171960
171960
171960
2866.0


In [11]:
df = pd.DataFrame({'pessoa': np.repeat(np.arange(1, 2867), 60), 'tempo': np.tile(np.arange(1,61), 2866), 'atividade': atividade, 'acelerômetro': dadospa, 'giroscópio': dadospg})

In [12]:
df.to_csv('base_de_dados', index=False)